# HUK Coding Challenge

Die Aufgabe besteht in der Modellierung einer Kundenaffinität zum Abschluss einer KFZ-Versicherung.

In [1]:
import os
from datetime import datetime
import logging
import csv
import pickle

import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from functools import reduce
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import RandomOverSampler

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)-15s %(message)s')
logger = logging.getLogger()

In [3]:
def detect_delimiter(filename):
    """
    This function looks for the delimiter in a file.

    Inputs:
        - filename (str): path to specific file
    Returns:
        - delimiter (str)
    """
    with open(filename, 'r', newline='') as file:
        dialect = csv.Sniffer().sniff(file.read(1024))
        return dialect.delimiter

## Loading Data

In [4]:
input_folder_path = 'data/input_data/'
directories = [os.path.join(os.getcwd(), '..', input_folder_path)]
file_list = []
dataframes = []

# search in all specified directories
for directory in directories:
    # list content of directory
    file_names = os.listdir(os.path.join(os.getcwd(), '..', directory))
    logger.info(f'Found files: {file_names}')
    for each_file_name in file_names:
        file_list.append(each_file_name)
        # get filepath to relevant files
        file_path = os.path.join(os.getcwd(), '..', directory, each_file_name)
        # error handling for the one file using a different delimiter
        delimiter = detect_delimiter(file_path)
        if delimiter == ',':
            current_df = pd.read_csv(file_path)
            dataframes.append(current_df)
        else:
            current_df = pd.read_csv(file_path, delimiter=';')
            dataframes.append(current_df)

# Merge all dataframes into one
df_merged = reduce(lambda left, right: pd.merge(left, right, on='id'), dataframes)

# Deduplicate
df_merged = df_merged.drop_duplicates()

# Check wether data path exists
if not os.path.exists('../data/raw_data/'):
    os.makedirs('../data/raw_data/')

# save merged dataframe as csv
df_merged.to_csv('../data/raw_data/raw_data.csv')

2023-05-21 17:26:10,020 Found files: ['rest.csv', 'interesse.csv', 'alter_geschlecht.csv']


## Explorative Datenanalyse (EDA)

Machen Sie sich mit dem Datensatz vertraut. Identifizieren Sie dabei mögliche Probleme sowie grundlegende statistische Zusammenhänge, welche für die anschließende Modellierung wichtig sein könnten.

In [5]:
df_raw = df_merged.copy()

In [6]:
profile = ProfileReport(df_raw, title='Pandas Profiling Report on raw data')
# open report from output.html file generated from this cell
profile.to_file("../eda_output.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/philipherp/opt/anaconda3/envs/huk_coding_challenge/lib/python3.10/site-packages/multimethod/__init__.py:184: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  return self[tuple(map(self.get_type, args))](*args, **kwargs)


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
# Count values in column Interesse --> Imbalanced Dataset
df_raw['Interesse'].value_counts()

0.0    334399
1.0     46710
Name: Interesse, dtype: int64

In [8]:
df_raw.head()

,Fahrerlaubnis,Regional_Code,Vorversicherung,Alter_Fzg,Vorschaden,Jahresbeitrag,Vertriebskanal,Kundentreue,id,Interesse,Geschlecht,Alter
0,1,15.0,1,1-2 Year,No,2630.0,124.0,74,317635,0.0,Male,76
1,1,28.0,0,1-2 Year,Yes,2630.0,125.0,213,337993,0.0,Male,43
2,1,33.0,0,1-2 Year,Yes,27204.0,124.0,114,160325,0.0,Male,20
3,1,46.0,1,< 1 Year,No,31999.0,152.0,251,141620,0.0,Male,24
4,1,49.0,0,1-2 Year,Yes,28262.0,26.0,60,75060,0.0,Male,51


### Cleaning Data

- Number of variables:      12
- Number of observations:   381109
- Missing cells:            0
- Missing cells %:          0%
- Duplicate rows:           0
- Categorical:              2
- Numeric:                  8
- Boolean:                  1
- Variables:
    - Fahrerlaubnis:                Highly imbalanced (97.8%)
    - Vertriebskanal > Alter:       High correlation
    - Vorversicherung > Vorschaden: High correlation
    - Alter_Fzg > Vertriebskanal:   High correlation
    - id:                           uniformly distributed & unique values

In [9]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 381109 entries, 0 to 381108
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Fahrerlaubnis    381109 non-null  int64  
 1   Regional_Code    381109 non-null  float64
 2   Vorversicherung  381109 non-null  int64  
 3   Alter_Fzg        381109 non-null  object 
 4   Vorschaden       381109 non-null  object 
 5   Jahresbeitrag    381109 non-null  float64
 6   Vertriebskanal   381109 non-null  float64
 7   Kundentreue      381109 non-null  int64  
 8   id               381109 non-null  int64  
 9   Interesse        381109 non-null  float64
 10  Geschlecht       381109 non-null  object 
 11  Alter            381109 non-null  int64  
dtypes: float64(4), int64(5), object(3)
memory usage: 45.9+ MB


In [19]:
# Check wether data path exists
if not os.path.exists('../data/cleaned_data/'):
    os.makedirs('../data/cleaned_data/')

# save merged dataframe as csv
df_raw.to_csv('../data/cleaned_data/cleaned_data.csv')
df_cleaned = df_raw.copy()

## Feature Engineering

Bereiten Sie, soweit für Ihre Modellierung nötig, die Variablen geeignet auf.

In [20]:
# Get column names
column_names = df_cleaned.columns

# Get categorical columns
categorical_columns = df_cleaned.select_dtypes(include="object").columns

# Get numerical columns
#numerical_columns = df_cleaned.select_dtypes(include="number").columns

# Create pipeline for categorical columns
categorical_pipeline = Pipeline([
    ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Create transformer for all columns
preprocessor = ColumnTransformer([
    ('categorical_pipeline', categorical_pipeline, categorical_columns)
], remainder='passthrough')

# Fit and transform data
df_cleaned = preprocessor.fit_transform(df_cleaned)

# Convert to dataframe
df_cleaned = pd.DataFrame(df_cleaned, columns=['lag_1', 
                                               'lag_2', 
                                               'lag_3', 
                                               'lag_4', 
                                               'lag_5', 
                                               'lag_6', 
                                               'lag_7', 
                                               'Fahrerlaubnis', 
                                               'Regional_Code', 
                                               'Vorversicherung', 
                                               'Jahresbeitrag', 
                                               'Vertriebskanal', 
                                               'Kundentreue', 
                                               'id', 
                                               'Interesse', 
                                               'Alter'])

# Put id column as first column
# and Interesse as last column
df_cleaned = df_cleaned[['id', 
                         'lag_1', 
                         'lag_2', 
                         'lag_3', 
                         'lag_4', 
                         'lag_5', 
                         'lag_6', 
                         'lag_7', 
                         'Fahrerlaubnis', 
                         'Regional_Code', 
                         'Vorversicherung', 
                         'Jahresbeitrag', 
                         'Vertriebskanal', 
                         'Kundentreue', 
                         'Alter', 
                         'Interesse']]                                             

# change unnecessary floats to int
float_columns = ['Fahrerlaubnis', 
                 'Regional_Code', 
                 'Vorversicherung', 
                 'Vertriebskanal', 
                 'Kundentreue', 
                 'Alter', 
                 'Interesse']
df_cleaned[float_columns] = df_cleaned[float_columns].astype('int64')

# Check wether data path exists
if not os.path.exists('../data/encoded_data/'):
    os.makedirs('../data/encoded_data/')

# save merged dataframe as csv
df_cleaned.to_csv('../data/encoded_data/encoded_data.csv')
df_encoded = df_cleaned.copy()

In [12]:
df_encoded.head()

,id,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,Fahrerlaubnis,Regional_Code,Vorversicherung,Jahresbeitrag,Vertriebskanal,Kundentreue,Alter,Interesse
0,317635.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1,15,1,2630.0,124,74,76,0
1,337993.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1,28,0,2630.0,125,213,43,0
2,160325.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1,33,0,27204.0,124,114,20,0
3,141620.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1,46,1,31999.0,152,251,24,0
4,75060.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1,49,0,28262.0,26,60,51,0


In [21]:
# Drop features from df_encode with feature importance below 0.05
# Also drop Features Vertriebskanal and Vorversicherung, since they highly correlate with more important features
df_encoded = df_encoded.drop(['lag_1',
                              'lag_2',
                              'lag_3',
                              'lag_6',
                              'lag_7',
                              'Fahrerlaubnis',
                              'Vertriebskanal',
                              'Vorversicherung',
                              'id'], axis=1)
df_encoded.head()

,id,lag_4,lag_5,Regional_Code,Jahresbeitrag,Kundentreue,Alter,Interesse
0,317635.0,1.0,0.0,15,2630.0,74,76,0
1,337993.0,0.0,1.0,28,2630.0,213,43,0
2,160325.0,0.0,1.0,33,27204.0,114,20,0
3,141620.0,1.0,0.0,46,31999.0,251,24,0
4,75060.0,0.0,1.0,49,28262.0,60,51,0


## Modellvergleich

Entscheiden Sie sich für ein geeignetes Modell zur Prognose der Kundenaffinität. Erläutern Sie wie Sie dabei vorgehen und begründen Sie Ihre Entscheidung.

__Logistische Regression:__

Die logistische Regression ist eine weit verbreitete Methode zur Vorhersage von binären Ergebnissen. Sie modelliert die Wahrscheinlichkeit, dass eine Beobachtung einer bestimmten Klasse angehört, basierend auf einer Kombination von Eingangsvariablen. Das Modell kann dann die Wahrscheinlichkeit schätzen, dass ein Kunde affin oder nicht affin ist und eine Entscheidungsgrenze festlegen, um die Vorhersage zu treffen.

__Random Forest Classifier:__

Ein Random Forest Classifier ist ein Ensemble-Modell, das aus mehreren Entscheidungsbäumen besteht. Jeder Baum wird auf einem zufälligen Teil des Datensatzes trainiert, und die Vorhersage erfolgt durch Abstimmung der Vorhersagen der einzelnen Bäume. Random Forests sind in der Regel robust gegenüber Overfitting und können gut mit einer Mischung aus kategorischen und numerischen Variablen umgehen. Sie können auch die wichtigsten Merkmale identifizieren, die zur Vorhersage beitragen.

__Gradient Boosting-Modelle:__

Gradient Boosting-Modelle wie der Gradient Boosting Classifier oder der XGBoost Classifier sind ensemblebasierte Modelle, die durch die Kombination mehrerer schwacher Lernalgorithmen starke Vorhersagemodelle erstellen. Sie sind bekannt für ihre hohe Vorhersagegenauigkeit.

__Neuronale Netzwerke:__

Neuronale Netzwerke sind leistungsstarke Modelle, die in der Lage sind, komplexe Zusammenhänge in den Daten zu erfassen.

#### Hauptvorteile von RF::

- Random Forests sind in der Regel robuster gegenüber Overfitting als logistische Regressionen, da sie mehrere Entscheidungsbäume verwenden, um eine Vorhersage zu treffen, anstatt nur einen einzigen Entscheidungsbaum zu verwenden.
- Random Forests können gut mit einer Mischung aus kategorischen und numerischen Variablen umgehen, während logistische Regressionen nur numerische Variablen verarbeiten können.
- Random Forests können die wichtigsten Merkmale identifizieren, die zur Vorhersage beitragen, während logistische Regressionen nur die Koeffizienten der einzelnen Merkmale liefern können.
- Robustheit gegenüber Ausreißern: Aggregationen mehrerer Entscheidungsbäume reduziert den Einfluss von Ausreißern.
- Nichtlinearität: Random Forests sind nicht parametrisch und können daher nichtlineare Zusammenhänge zwischen Merkmalen und Zielvariablen modellieren.

## Modellbuilding

1. Trainieren Sie das von Ihnen gewählte Modell. Wählen Sie geeignete Metriken um die Güte des finalen Modells zu beurteilen.
2. Zeigen Sie, welche Variablen und Zusammenhänge für Ihre finales Modell relevant sind.
3. Überlegen Sie sich (ohne Umsetzung) wie Sie Ihr Modell weiter optimieren können.

In [15]:
# Train a random forest classifier model 
# and select suitable hyperparameters
# and metrics to view the models performance

# Split data into train and test set
X_train, X_test, y_train, y_test = train_test_split(df_encoded.drop(['Interesse'], axis=1),
                                                    df_encoded['Interesse'],
                                                    test_size=0.3, 
                                                    random_state=42)

# Use random oversampler to balance dataset on Interesse = 1
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

# Train model
model = RandomForestClassifier(random_state=42)
model.fit(X_resampled, y_resampled)

y_train = pd.DataFrame(y_train, columns=['Interesse'])

# Predict on train set
y_train_pred = model.predict(X_train)

# Evaluate model on train set
logger.info(f'Train Accuracy: {accuracy_score(y_train, y_train_pred)}')
logger.info(f'Train Precision: {precision_score(y_train, y_train_pred)}')
logger.info(f'Train Recall: {recall_score(y_train, y_train_pred)}')
logger.info(f'Train F1: {f1_score(y_train, y_train_pred)}')

2023-05-21 17:27:39,993 Train Accuracy: 0.9999978639690019
2023-05-21 17:27:40,094 Train Precision: 1.0
2023-05-21 17:27:40,195 Train Recall: 0.9999957248697154
2023-05-21 17:27:40,296 Train F1: 0.9999978624302884


### Test the models performance on test data

In [16]:
# Predict on test set
y_pred = model.predict(X_test)

# Evaluate model
logger.info(f'Test Accuracy: {accuracy_score(y_test, y_pred)}')
logger.info(f'Test Precision: {precision_score(y_test, y_pred)}')
logger.info(f'Test Recall: {recall_score(y_test, y_pred)}')
logger.info(f'Test F1: {f1_score(y_test, y_pred)}')

2023-05-21 17:27:44,541 Test Accuracy: 0.9450309011164274
2023-05-21 17:27:44,586 Test Precision: 0.9040568739216448
2023-05-21 17:27:44,631 Test Recall: 0.9959398137090996
2023-05-21 17:27:44,676 Test F1: 0.9477766360937359


In [ ]:
# Check wether data path exists
if not os.path.exists('../models/'):
    os.makedirs('../models/')

# Save model with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

with open(f'../models/cv_model_{timestamp}.pkl', 'wb') as file:
    pickle.dump(model, file)

#### Train another model using k-fold cross-validation

In [28]:
# Train a Random Forest Classifier model using k-fold cross validation
# and select suitable hyperparameters
# and metrics to view the models performance

from sklearn.model_selection import cross_validate, KFold

# Create the Random Forest Classifier model
cv_model = RandomForestClassifier(random_state=42)

# Define the number of folds for cross-validation
k = 5

# Perform k-fold cross-validation
cv = KFold(n_splits=k, shuffle=True, random_state=42)

scoring = ['accuracy', 'precision', 'recall', 'f1']

# Perform cross-validation
scores = cross_validate(model, X_resampled, y_resampled, cv=k, scoring=scoring)

# Fit the model on the entire dataset
model.fit(X_resampled, y_resampled)

# Calculate the average scores across all folds
mean_accuracy = scores['test_accuracy'].mean()
mean_precision = scores['test_precision'].mean()
mean_recall = scores['test_recall'].mean()
mean_f1 = scores['test_f1'].mean()

# Print the average scores
print("Average Scores:")
print("Accuracy:", mean_accuracy)
print("Precision:", mean_precision)
print("Recall:", mean_recall)
print("F1-Score:", mean_f1)



Average Scores:
Accuracy: 0.9568778605484705
Precision: 0.9209845098017443
Recall: 0.9995095693779904
F1-Score: 0.9586411129263478


Accuracy Score:
Accuracy_score berechnet den Prozentsatz der vom Modell getroffenen korrekten Vorhersagen aus allen Vorhersagen. Sie wird berechnet, indem die Anzahl der richtigen Vorhersagen durch die Gesamtzahl der Vorhersagen dividiert wird. Eine hohe Genauigkeitsbewertung weist auf eine insgesamt hohe Vorhersageleistung des Modells hin. Allerdings reicht die Genauigkeit allein möglicherweise nicht aus, wenn der Datensatz unausgeglichen ist.

Precision Score:
Precision_score misst den Anteil korrekt vorhergesagter positiver Instanzen an allen als positiv vorhergesagten Instanzen. Sie wird berechnet, indem die Anzahl der wahr-positiven Ergebnisse durch die Summe der wahr-positiven und falsch-positiven Ergebnisse dividiert wird. Präzision konzentriert sich auf die Qualität positiver Vorhersagen. Eine hohe Präzisionsbewertung weist auf eine niedrige Falsch-Positiv-Rate hin, was bedeutet, dass das Modell, wenn es eine positive Klasse vorhersagt, wahrscheinlich richtig ist.

Recall Score:
Recall_score, auch bekannt als Sensitivität oder True-Positive-Rate, misst den Anteil korrekt vorhergesagter positiver Instanzen an allen tatsächlich positiven Instanzen. Sie wird berechnet, indem die Anzahl der wahr-positiven Ergebnisse durch die Summe der wahr-positiven und falsch-negativen Ergebnisse dividiert wird. Recall konzentriert sich auf die Fähigkeit des Modells, alle positiven Instanzen zu finden, ohne welche zu übersehen. Ein hoher Erinnerungswert weist auf eine niedrige Falsch-Negativ-Rate hin, was bedeutet, dass das Modell einen großen Anteil positiver Fälle korrekt identifizieren kann.

F1 Score:
Der F1_score ist das harmonische Mittel aus Präzision und Erinnerung. Es bietet ein Gleichgewicht zwischen Präzision und Erinnerung und ist nützlich, wenn Sie sowohl falsch-positive als auch falsch-negative Ergebnisse berücksichtigen möchten. Sie wird wie folgt berechnet: 2 * ((Präzision * Rückruf) / (Präzision + Rückruf)). Der F1-Score ist eine einzelne Metrik, die Präzision und Erinnerung kombiniert. Ein hoher F1-Score weist auf eine gute Gesamtleistung hin, wenn man sowohl Präzision als auch Rückruf berücksichtigt.

Zusammenfassend lässt sich sagen, dass Accuracy_score ein Gesamtleistungsmaß bietet, während Precision_score, Recall_score und f1_score Einblicke in verschiedene Aspekte der Modellleistung bieten. Berücksichtigen Sie Ihre spezifischen Anforderungen und die Art Ihres Problems, um zu bestimmen, welche Metrik(en) für Ihre Bewertung am wichtigsten sind.

In [ ]:
# Check wether data path exists
if not os.path.exists('../models/'):
    os.makedirs('../models/')

# Save model with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

with open(f'../models/cv_model_{timestamp}.pkl', 'wb') as file:
    pickle.dump(model, file)

## Possible strategies to optimize the Random Forrest Model

#### __Feature Selection:__

Bewerten Sie die Bedeutung jedes Features in Ihrem Modell und erwägen Sie, weniger informative oder stark korrelierte Features zu entfernen. Dies kann durch die Untersuchung der Feature-Wichtigkeiten erreicht werden, die das Random-Forest-Modell liefert.

In [ ]:
# Identify the most important features
feature_importances = pd.DataFrame(model.feature_importances_, index=X_train.columns, columns=['importance']).sort_values('importance', ascending=False)
feature_importances

,importance
id,0.182870
lag_4,0.169762
Kundentreue,0.158571
lag_5,0.152979
Jahresbeitrag,0.142746
Alter,0.130988
Regional_Code,0.062085


#### __Recursive Feature Elimination (RFE):__

Rekursive Feature-Eliminierung (RFE) ist eine Feature-Auswahlmethode, die ein Modell anpasst und das schwächste Feature (oder die schwächsten Features) entfernt, bis die angegebene Anzahl von Features erreicht ist. Es basiert auf der Idee, dass gute Vorhersagemodelle nicht unbedingt alle Funktionen erfordern. RFE kann verwendet werden, um die Modellleistung zu verbessern und Überanpassungen zu reduzieren, indem eine Teilmenge von Merkmalen ausgewählt wird, die für die Zielvariable am relevantesten sind.

#### __Hyperparameter Tuning:__ 

Zufällige Wälder verfügen über verschiedene Hyperparameter, die angepasst werden können, um die Leistung zu verbessern und Überanpassungen zu reduzieren. Zu den wichtigsten Hyperparametern gehören die Anzahl der Bäume (n_estimators), die maximale Tiefe der Bäume (max_ Depth) und die minimale Anzahl von Stichproben, die zum Teilen eines internen Knotens erforderlich sind (min_samples_split). Mithilfe von Techniken wie der Rastersuche oder der Zufallssuche können Sie optimale Hyperparameterwerte finden, die Modellkomplexität und Leistung in Einklang bringen.

#### Correlation Analysis

Eine Analyse der Korrelation zwischen den Merkmalen und der Zielgröße kann helfen, redundante oder stark korrelierte Merkmale zu identifizieren. In solchen Fällen kann eines der korrelierten Merkmale entfernt werden, um eine Überanpassung zu vermeiden.

#### __Cross-Validation:__ 

Anstatt das Modell nur anhand des Trainingssatzes zu bewerten, wird eine Kreuzvalidierung durchgeführt, um seine Leistung bei mehreren train-test-splits zu bewerten. Dies trägt zu einer zuverlässigeren Schätzung der Leistung des Modells bei und kann Aufschluss darüber geben, ob eine Überanpassung vorliegt.

Der Vorteil der Kreuzvalidierung besteht darin, dass sie im Vergleich zu einem einzelnen train-test-split eine robustere Schätzung der Modellleistung liefert. Es hilft bei der Beurteilung der Fähigkeit des Modells, auf unsichtbare Daten zu verallgemeinern, und verringert den Einfluss der spezifischen Datenaufteilung auf die Bewertung.
Um eine Kreuzvalidierung in scikit-learn durchzuführen, können Sie die Funktion „cross_val_score“ oder die Funktion „cross_validate“ verwenden und dabei die Anzahl der Faltungen (cv-Parameter) und die gewünschte Bewertungsmetrik angeben. Diese Funktionen übernehmen die Datenaufteilung und Modellauswertung automatisch und erleichtern so die Durchführung einer Kreuzvalidierung mit zufälligen Gesamtstrukturen.

#### __Increase Training Data:__ 

Erhalten Sie nach Möglichkeit mehr Trainingsdaten, um eine umfassendere Darstellung der zugrunde liegenden Muster zu erhalten. Ein größerer Datensatz kann dazu beitragen, das Modell besser zu verallgemeinern und eine Überanpassung zu reduzieren.

## Addendum

In [ ]:
# Train a random forest classifier model
# select suitable hyperparamters with grit search
# select metrics to view the models performance

# Get categorical columns
categorical_columns = df_cleaned.select_dtypes(include="object").columns

# Get numerical columns
numerical_columns = df_cleaned.select_dtypes(include="number").columns

# Create pipeline for categorical columns
categorical_pipeline = Pipeline([
    ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Create transformer for all columns
preprocessor = ColumnTransformer([
    ('categorical_pipeline', categorical_pipeline, categorical_columns)
], remainder='passthrough')

# Fit and transform data
df_cleaned = preprocessor.fit_transform(df_cleaned)

# Convert to dataframe
df_cleaned = pd.DataFrame(df_cleaned, columns=['lag_1', 
                                               'lag_2', 
                                               'lag_3', 
                                               'lag_4', 
                                               'lag_5', 
                                               'lag_6', 
                                               'lag_7', 
                                               'Fahrerlaubnis', 
                                               'Regional_Code', 
                                               'Vorversicherung', 
                                               'Jahresbeitrag', 
                                               'Vertriebskanal', 
                                               'Kundentreue', 
                                               'id', 
                                               'Interesse', 
                                               'Alter'])

# Put id column as first column
# and Interesse as last column
df_cleaned = df_cleaned[['id', 
                         'lag_1', 
                         'lag_2', 
                         'lag_3', 
                         'lag_4', 
                         'lag_5', 
                         'lag_6', 
                         'lag_7', 
                         'Fahrerlaubnis', 
                         'Regional_Code', 
                         'Vorversicherung', 
                         'Jahresbeitrag', 
                         'Vertriebskanal', 
                         'Kundentreue', 
                         'Alter', 
                         'Interesse']]

# change unnecessary floats to int
float_columns = ['Fahrerlaubnis', 
                 'Regional_Code', 
                 'Vorversicherung', 
                 'Vertriebskanal', 
                 'Kundentreue', 
                 'Alter']
df_cleaned[float_columns] = df_cleaned[float_columns].astype('int64')

df_encoded = df_cleaned.copy()

# Train a random forest classifier model
# select suitable hyperparamters with grit search
# select metrics to view the models performance

# train, test, split
# Split data into train and test
X = df_encoded.drop(['Interesse'], axis=1)
y = df_encoded['Interesse']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the parameter grid based on the results of random search
param_grid = {
    'bootstrap': [True],
    'max_depth': [10, 20, 30, 40, 50],
    'max_features': [2, 3, 4, 5, 6, 7, 8, 9, 10],
    'min_samples_leaf': [3, 4, 5, 6, 7],
    'min_samples_split': [8, 10, 12, 14, 16],
    'n_estimators': [100, 200, 300, 400, 500]
}

# Create a based model
rf = RandomForestClassifier()

# Instantiate the grid search model
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid,
                            cv = 3, n_jobs = -1, verbose = 2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get best parameters
grid_search.best_params_

# Get best estimator
best_grid = grid_search.best_estimator_

# Get predictions
y_pred = best_grid.predict(X_test)

# Get metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label=2)
recall = recall_score(y_test, y_pred, pos_label=2)
f1 = f1_score(y_test, y_pred, pos_label=2)

# Print metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1: {f1}')

# Check wether data path exists
if not os.path.exists('../models/'):
    os.makedirs('../models/')

# save model
pickle.dump(best_grid, open('../models/model.pkl', 'wb'))

# save preprocessor
pickle.dump(preprocessor, open('../models/preprocessor.pkl', 'wb'))

# save metrics
metrics = {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}
pickle.dump(metrics, open('../models/metrics.pkl', 'wb'))

# save predictions
predictions = {'y_pred': y_pred}
pickle.dump(predictions, open('../models/predictions.pkl', 'wb'))

Fitting 3 folds for each of 5625 candidates, totalling 16875 fits
[CV] END bootstrap=True, max_depth=10, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100; total time=  18.8s
[CV] END bootstrap=True, max_depth=10, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100; total time=  18.8s
[CV] END bootstrap=True, max_depth=10, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100; total time=  19.1s


KeyboardInterrupt: 